# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   562.756560  0.447770  1234  563.204330  562.308790  1126.408660   
6   646.211222  0.375973  1234  646.587195  645.835248  1293.174390   
7   390.483033  0.808333  1234  391.291366  389.674700   782.582733   
8   729.552474 -0.115363  1234  729.437111  729.667836  1458.874223   
9   359.764473  0.609246  1234  360.373718  359.155227   720.747437   
10  888.909196  0.380660  1234  889.289855  888.528536  1778.579711   
11  419.543222 -0.808070  1234  418.735152  420.351292   837.470305   
12  980.369087  0.984459  1234  981.353546  979.384628  1962.707092   
13  360.784941 -0.988090  1234  359.796851  361.773031   719.593703   
14  195.888832 -0.111413  1234  195.777419  196.000245   391.554838   

              f  
5   1124.617579  
6   1291.670496  
7    779.349399  
8   1459.335673  
9    718.310454  
10  1777.057072  
11   840.702584  
12  1958.769257  
13   723.546063  
14   392.000489

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -563.204330   562.308790
6   -646.587195   645.835248
7   -391.291366   389.674700
8   -729.437111   729.667836
9   -360.373718   359.155227
10  -889.289855   888.528536
11  -418.735152   420.351292
12  -981.353546   979.384628
13  -359.796851   361.773031
14  -195.777419   196.000245

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -563.204330   562.308790
6   -646.587195   645.835248
7   -391.291366   389.674700
8   -729.437111   729.667836
9   -360.373718   359.155227
10  -889.289855   888.528536
11  -418.735152   420.351292
12  -981.353546   979.384628
13  -359.796851   361.773031
14  -195.777419   196.000245

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-563.20433009,  562.30878954],
       [-646.58719493,  645.83524821],
       [-391.29136642,  389.67469959],
       [-729.43711129,  729.66783634],
       [-360.37371849,  359.15522686],
       [-889.28985549,  888.52853587],
       [-418.7351524 ,  420.35129186],
       [-981.35354609,  979.3846284 ],
       [-359.79685134,  361.77303127],
       [-195.7774192 ,  196.00024452]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1124.408660  -1124.617579
6    1291.174390  -1291.670496
7     780.582733   -779.349399
8    1456.874223  -1459.335673
9     718.747437   -718.310454
10   1776.579711  -1777.057072
11    835.470305   -840.702584
12   1960.707092  -1958.769257
13    717.593703   -723.546063
14    389.554838   -392.000489

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf